In [4]:
import pandas as pd
import numpy as np
import math
import json 
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
import re
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import warnings
warnings.filterwarnings('ignore')
search = SearchEngine()

#os.chdir("D:\CSP 571\yelp data")


In [5]:
business_data = pd.read_csv("Business_in_Illinois.csv") 
business_data.columns.values[0] = "index"

In [6]:
business_data.head()

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,mofOjB6flg-eAWOFbOkHfQ,ChinaTown Buffet,713 W Marketview Dr,Champaign,IL,61822.0,40.137270,-88.256043,2.5,72,1,"{'WiFi': ""u'no'"", 'RestaurantsReservations': '...",Restaurants,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
1,1,m4DwPVxmFDQE_FnGIOa3cQ,Sunny China Buffet,1703 Philo Rd,Urbana,IL,61802.0,40.097639,-88.191217,3.5,15,1,"{'RestaurantsGoodForGroups': 'True', 'Business...",Restaurants,NaN
2,2,HbK6IfznbVToEaKVC0WcSg,The Clark Bar,207 W Clark St,Champaign,IL,61820.0,40.115017,-88.246156,3.5,24,1,"{'Alcohol': ""u'full_bar'"", 'GoodForKids': 'Fal...",Restaurants,"{'Tuesday': '16:0-2:0', 'Wednesday': '16:0-2:0..."
3,3,o9nGD3J0btXjE9M6IdgZTA,McAlister's Deli,421 Town Center Rd,Champaign,IL,61820.0,40.141668,-88.250360,3.0,50,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '10:30-21:0',..."
4,4,sDfN5qJlmvrwTfwEPBsxSg,Dairy Queen,1103 Klein Ave,Rantoul,IL,61866.0,40.311299,-88.142882,2.0,4,1,"{'GoodForKids': 'False', 'BusinessAcceptsCredi...",Restaurants,NaN


In [7]:
business_data.shape

(457, 15)

In [4]:
## check missing values
missing_value_count = business_data.isnull().sum()
print('Missing Count ', missing_value_count)

## Fill empty postal code
def get_postal_code(col_values):
    if (pd.isna(col_values['postal_code'])):
        zip_code = search.by_coordinates(col_values['latitude'], col_values['longitude'], radius=10)
        col_values['postal_code'] = zip_code[0].zipcode
    return col_values
business_data = business_data.apply(get_postal_code, axis = 1)


Missing Count  index            0
business_id      0
name             0
address          4
city             0
state            0
postal_code      1
latitude         0
longitude        0
stars            0
review_count     0
is_open          0
attributes      18
categories       0
hours           87
dtype: int64


#### Since the missing values of address, attributes and hours correspond to missing entry, we cannot comprehend their values from other columns


In [5]:
business_data['hours'] = business_data['hours'].fillna('')

business_data.isnull().sum()

index            0
business_id      0
name             0
address          4
city             0
state            0
postal_code      0
latitude         0
longitude        0
stars            0
review_count     0
is_open          0
attributes      18
categories       0
hours            0
dtype: int64

In [7]:
## Update hours with missing values for entire week 
def hours_check(col_values):
    pattern = "{('\w+':'0:0-0:0'){7}}"
    matched = None
    result = json.dumps(col_values['hours'])
    result = result.replace(" ", "")
    result = result.replace(",", "")
    matched = re.search(pattern, result)
    if (matched):
        col_values['hours'] = 0
        
    return col_values
    
business_data = business_data.apply(hours_check, axis = 1)

In [8]:
business_data.head()

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,mofOjB6flg-eAWOFbOkHfQ,ChinaTown Buffet,713 W Marketview Dr,Champaign,IL,61822,40.137270,-88.256043,2.5,72,1,"{'WiFi': ""u'no'"", 'RestaurantsReservations': '...",Restaurants,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
1,1,m4DwPVxmFDQE_FnGIOa3cQ,Sunny China Buffet,1703 Philo Rd,Urbana,IL,61802,40.097639,-88.191217,3.5,15,1,"{'RestaurantsGoodForGroups': 'True', 'Business...",Restaurants,
2,2,HbK6IfznbVToEaKVC0WcSg,The Clark Bar,207 W Clark St,Champaign,IL,61820,40.115017,-88.246156,3.5,24,1,"{'Alcohol': ""u'full_bar'"", 'GoodForKids': 'Fal...",Restaurants,"{'Tuesday': '16:0-2:0', 'Wednesday': '16:0-2:0..."
3,3,o9nGD3J0btXjE9M6IdgZTA,McAlister's Deli,421 Town Center Rd,Champaign,IL,61820,40.141668,-88.250360,3.0,50,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '10:30-21:0',..."
4,4,sDfN5qJlmvrwTfwEPBsxSg,Dairy Queen,1103 Klein Ave,Rantoul,IL,61866,40.311299,-88.142882,2.0,4,1,"{'GoodForKids': 'False', 'BusinessAcceptsCredi...",Restaurants,


In [9]:
## Checking for inconsistent Data
cities = business_data['city'].unique()
cities

array(['Champaign', 'Urbana', 'Rantoul', 'Saint Joseph', 'Monticello',
       'Mahomet', 'Ogden', 'Savoy', 'St Joseph', 'Sidney', 'Tolono',
       'Philo', 'Fisher', 'Villa Grove', 'Homer', 'Mansfield', 'Gifford',
       'Tuscola'], dtype=object)

In [12]:
## There only city with inconsistent name is Saint Joseph, so we will update that
business_data['city'] = business_data['city'].str.lower()
business_data['city'] = business_data['city'].str.strip()

def replace_matches(df, column, string_to_match, min_ratio = 80):
    strings = df[column].unique()
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match
    
for city in cities:
    replace_matches(business_data, column='city', string_to_match=city)

business_data['city'].unique()  

array(['Champaign', 'Urbana', 'Rantoul', 'St Joseph', 'Monticello',
       'Mahomet', 'Ogden', 'Savoy', 'Sidney', 'Tolono', 'Philo', 'Fisher',
       'Villa Grove', 'Homer', 'Mansfield', 'Gifford', 'Tuscola'],
      dtype=object)

In [13]:
business_data.shape

(457, 15)

In [ ]:
business_data.to_csv('Business_in_Illinois.csv')